In [1]:
import numpy as np
from gaussian_approximation import get_pte_from_gaussian
from kde_method import get_pte_KDE
from nearest_neighbour_method import get_pte_nearest_neighbour
from stats import *

One dimensional test case, x0 is at 1 sigma

In [2]:
nsamples = int(1e5)

x0 = 2
mean = 0
cov = 1

In [3]:
pte = get_pte_from_gaussian(x0, mean, cov, nsamples)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

PTE = 0.04527000000000003
Number of sigma = 2.0021369916974976


Two dimensional test case

In [4]:
x0 = np.array([np.sqrt(6),0])
mean = (0, 0)
cov = [[1, 0], [0, 1]]

In [5]:
pte = get_pte_from_gaussian(x0, mean, cov, nsamples)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

PTE = 0.049900000000000055
Number of sigma = 1.960820206837927


Compare with KDE method from a chain with the same covariance and means

In [6]:
path_to_chains = '/Users/Pablo/Code/PolyChordLite_old/chains/gauss'

In [ ]:
pte = get_pte_KDE(x0, path_to_chains, bandwidth = 0.01)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

In [ ]:
pte = get_pte_nearest_neighbour(x0, path_to_chains)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

Now try with the DES and Planck case. First, the Gaussian approximation. We can build a covariance from both covariances

In [ ]:
#DES case
path_to_chains = '/Users/Pablo/Projets/Tension/diff_chains/diff_full'

means_planck = np.loadtxt('/Users/Pablo/Projets/Tension/sims_mean_var/baseline/cosmological_parameters/values.txt', usecols = [2], unpack=True)[[17,3,13,10,0, -6]]
x1_s8 = np.loadtxt('/Users/Pablo/Projets/Tension/sims_mean_var/s8_1sigma/cosmological_parameters/values.txt', usecols = [2], unpack=True)[[17,3,13,10,0, -6]]
x0 = x1_s8 - means_planck
x0[4] = x0[4]*1e9

c_des = np.loadtxt('/Users/Pablo/Projets/Tension/chains/chains_poly/0.01_tolerance/d_noiseless/covmat.txt') 
c_planck = np.loadtxt('/Users/Pablo/Projets/Tension/chains/chains_poly/0.01_tolerance/planck_poly0.01/covmat.txt') 

c_des_temp = c_des[:6, :6]
c_des_temp[-1] = c_des[-3, :6]
c_des_temp[:,-1] = c_des[:6,-3]
c_des_temp[-1,-1] = c_des[-3,-3]
c_des = c_des_temp

c_des[4] = c_des[4]*1e9
c_des[:,4] = c_des[:,4]*1e9

c_planck_temp = c_planck[:6, :6]
c_planck_temp[-1] = c_planck[-3, :6]
c_planck_temp[:,-1] = c_planck[:6,-3]
c_planck_temp[-1,-1] = c_planck[-3,-3]
c_planck = c_planck_temp

c_planck[4] = c_planck[4]*1e9
c_planck[:,4] = c_planck[:,4]*1e9


mean = np.zeros(6)
cov = c_des+c_planck

nsamples = int(1e6)

In [ ]:
pte = get_pte_from_gaussian(x0, mean, cov, nsamples)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

To compare, we use the difference chain and the nearest neighbour method

In [ ]:
pte = get_pte_nearest_neighbour(x0, path_to_chains)
nsigma = get_nsigma(pte)

print('PTE =', pte)
print('Number of sigma =', nsigma)

In [1]:
from kde_method import *

In [3]:
path_to_chains = '/Users/Pablo/Code/PolyChordLite_old/chains/gauss'
weights, loglike_samples, X = load_chains(path_to_chains)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut

bandwidths = 10 ** np.linspace(-1, 1, 100)
grid = GridSearchCV(KernelDensity(kernel='gaussian'),
                    {'bandwidth': bandwidths},
                    cv=LeaveOneOut(len(X)))
grid.fit(X);

TypeError: LeaveOneOut() takes no arguments

In [13]:
grid = GridSearchCV(KernelDensity(kernel = 'gaussian'),{'bandwidth': np.linspace(0.1, 0.5, 20)}, cv = 5, iid = True)

In [15]:
grid.fit(X)

KeyboardInterrupt: 

In [25]:
bandwidths = 10 ** np.linspace(-3, 1, 50)
grid = GridSearchCV(KernelDensity(),
                    {'bandwidth': bandwidths},
                    cv=20) # 20-fold cross-validation
grid.fit(X[-10000:])
print(grid.best_params_)

{'bandwidth': 0.01151395399326447}


In [19]:
X.shape

(48531, 2)

In [20]:
X[-5000:]

array([[ 0.00995531,  0.02784858],
       [-0.02867025,  0.0150022 ],
       [-0.00323827, -0.01955982],
       ...,
       [-0.01551459, -0.01706476],
       [-0.03060693, -0.00200088],
       [-0.03575035, -0.01480529]])